In [2]:
import sys, os

def add_path(path):
    if path not in sys.path:
        sys.path.append(path)
        print(path, 'added')
    else:
        print(path, 'already exist')
# add_path('./lclibs')

import numpy as np
import pandas as pd
from scipy.stats import rankdata
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

from forecasters import *

def read_history(file):
    df = pd.read_csv(file, header=0)
    df['Date'] = pd.to_datetime(df['Date'])
    df.rename(columns={'Adjusted_close':'Adj Close'}, inplace=True)
    return df
    
# TODO: read data to be refined,use 'data_read.py'
data_dir = '../data'

# load history data
files = os.listdir(data_dir)
data = []
for file in files:
    stock = file.replace('.csv', '')
    file = os.path.join(data_dir, file)
    df = read_history(file)
    df['stock'] = stock.strip()
    data.append(df)

data = pd.concat(data)

# Fill in the missing dates
dates = data[['Date']].drop_duplicates()
dates['x'] = 1
stocks = data[['stock']].drop_duplicates()
stocks['x'] = 1
ds = dates.merge(stocks)
data = ds.merge(data, how='left').drop('x', axis=1).sort_values(['stock', 'Date']).fillna(method='ffill')

### add label
trading_dates = data['Date'].values
dates = pd.date_range(trading_dates.min(), trading_dates.max())
dates = pd.DataFrame(dates, columns=['date'])
dates['isin'] = dates['date'].isin(trading_dates)
dates['last_trading_date'] = dates['date']
dates.loc[~dates['isin'], 'last_trading_date'] = None
dates['last_trading_date'] = dates['last_trading_date'].fillna(method='ffill')

last_trading_date_dic = dict(zip(dates['date'], dates['last_trading_date']))

data['date_4w'] = data['Date'] + pd.Timedelta(days=28)  
data['date_4w'] = data['date_4w'].apply(lambda x:last_trading_date_dic.get(x, None)) # # 4周后上一个交易日
next_prices = data[['Date', 'stock', 'Adj Close']]
next_prices.columns = ['date_4w', 'stock', 'next_price']
data = data.merge(next_prices, how='left')

/home/notebook/code/group/intention_rec/lclibs already exist


In [3]:
data.stock.nunique()

100

In [4]:
returns = {}
for stock in data.stock.unique():
    tmp = data[data.stock == stock]
    tmp['next Close'] = tmp['Adj Close'].diff()#.iloc[1:].reset_index(drop=True)
    tmp = tmp.drop_duplicates(subset='Date', keep='first')
    return_single = tmp['next Close'] / tmp['Adj Close'].shift(1)
    if len(return_single) > 1287:
        print(stock)
    returns[stock] = return_single.iloc[1:].tolist()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
max([len(returns[key]) for key in returns.keys()])

1284

In [6]:
pd.DataFrame(returns).to_csv('real_returns_0501.csv', index=False)

In [7]:
(pd.DataFrame(returns) < -1).sum().sum()

0